In [1]:
"""
Politecnico di Torino - Laurea Triennale Ingegneria Energetica
Corso Energetica e Fonti Rinnovabili - Esercitazione Radiazione Solare, Solare Termico e Fotovoltaico
"""
__author__ = "Daniele Salvatore Schiera"
__version__ = "A.A. 2021/2022 - Rev. 05/04/2022"
__email__ = "daniele.schiera@polito.it"

# Imports
# for interactive
%matplotlib widget
# for printing
# %matplotlib inline 
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
from pvlib import solarposition, location
from pvlib.iotools import pvgis
from IPython.display import Markdown, Image, display, HTML
import ipywidgets as wds
plt.style.use('seaborn-whitegrid')
plt.ion()

# PV system calculation using PVGIS with pvlib
to use get_pvgis_hourly() https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.iotools.get_pvgis_hourly.html#pvlib.iotools.get_pvgis_hourly
and here the metadata of parameters of pvgis https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/getting-started-pvgis/api-non-interactive-service_en


In [2]:
pvgis.get_pvgis_hourly

<function pvlib.iotools.pvgis.get_pvgis_hourly(latitude, longitude, start=None, end=None, raddatabase=None, components=True, surface_tilt=0, surface_azimuth=0, outputformat='json', usehorizon=True, userhorizon=None, pvcalculation=False, peakpower=None, pvtechchoice='crystSi', mountingplace='free', loss=0, trackingtype=0, optimal_surface_tilt=False, optimalangles=False, url='https://re.jrc.ec.europa.eu/api/', map_variables=True, timeout=30)>

In [101]:
sunroofs = pd.read_csv("data/sunroofs.csv")
sunroofs.head()

,fid,slope_clas,fid_2,aspect_cla,slope_mean,A_ava,cos_aspect,sin_aspect,aspect_mea,id,_mean,xcoord,ycoord
0,38,2,40,2,24.703038,45.422640,-0.901033,0.400812,156.018792,1,0.900000,7.325949,45.753119
1,43,1,38,1,14.860566,16.559988,-0.469826,0.868352,118.415796,2,0.750000,7.321544,45.753066
2,124,1,133,2,6.699572,15.864334,-0.894905,0.119910,172.368265,3,1.000000,7.321129,45.752889
3,253,1,359,2,18.113563,26.576778,-0.939839,-0.028429,181.732605,4,0.666667,7.325872,45.752751
4,257,1,408,2,11.088444,22.681984,-0.860684,0.405279,154.785198,5,1.000000,7.326104,45.752737


In [102]:
rooftop = sunroofs.loc[sunroofs.id==1217] # 1217 1203 1195 1103 

In [126]:
L = rooftop.ycoord.values[0]
l_local = rooftop.xcoord.values[0] # west negative!
start = 2020 # last year available 2020
end = 2020
surface_tilt = 38 #rooftop.slope_mean.values[0]
surface_azimuth = 90 #rooftop.aspect_mea.values[0] - 180 # 0=south, west positive! 

#This is the power that the manufacturer declares that the PV array can produce under standard test conditions, 
#which are a constant 1000W of solar irradiance per square meter in the plane of the array, at an array temperature of 25°C. 
#The peak power should be entered in kilowatt-peak (kWp). 
#If you do not know the declared peak power of your modules but instead know the area of the modules (in m2) and the declared conversion efficiency (in percent), 
#you can calculate the peak power as power (kWp) = 1 kW/m2 * area * efficiency / 100. 
peakpower = 1 
# or
area_ava = np.ceil(rooftop.A_ava.values[0]*0.90) # m2
module_eff = 18 # %
peakpower = area_ava*module_eff/100

res = pvgis.get_pvgis_hourly(latitude=L,
                             longitude=l_local,
                             start=start,
                             end=end,
                             raddatabase='PVGIS-SARAH2',
                             surface_tilt=surface_tilt,
                             surface_azimuth=surface_azimuth,
                             outputformat='csv',
                             pvcalculation=True,
                             peakpower=peakpower,
                             pvtechchoice='crystSi',
                             loss=14,
                             url='https://re.jrc.ec.europa.eu/api/v5_2/',
                             map_variables = False)

In [127]:
parameters = res[1]
parameters

{'latitude': 45.736,
 'longitude': 7.325,
 'elevation': 574.0,
 'radiation_database': 'PVGIS-SARAH2',
 'Slope': '38 deg.',
 'Azimuth': '90 deg.',
 'Nominal power of the PV system (c-Si) (kWp)': '30.1',
 'System losses (%)': '14.0'}

In [128]:
timeseries = res[0]
timeseries.head(24)

,P,Gb(i),Gd(i),Gr(i),H_sun,T2m,WS10m,Int
time,,,,,,,,
2020-01-01 00:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-3.57,0.14,0
2020-01-01 01:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-2.20,0.21,0
2020-01-01 02:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-2.01,0.21,0
2020-01-01 03:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-1.90,0.14,0
2020-01-01 04:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-1.97,0.14,0
2020-01-01 05:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-2.11,0.14,0
2020-01-01 06:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-2.15,0.14,0
2020-01-01 07:10:00+00:00,0.00,0.00,0.00,0.00,0.00,-2.85,0.14,0
2020-01-01 08:10:00+00:00,777.05,0.00,41.67,1.06,7.21,-2.41,0.14,0


In [129]:
date = '2020-05-04'
date_end = '2020-05-04' #'2020-05-10'
plt.figure()
timeseries.loc[date:date_end].P.plot(legend='PV Power');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
plt.figure()
timeseries['Heq'] = timeseries.P/peakpower
timeseries['G(i)'] = timeseries['Gb(i)'] + timeseries['Gd(i)'] + timeseries['Gr(i)'] # W/m2
timeseries.loc[date:date_end]['Heq'].plot(legend='Heq')
timeseries.loc[date:date_end]['G(i)'].plot(legend='G(i)')
timeseries.loc[date:date_end]['Gb(i)'].plot(legend='Gb(i)')
timeseries.loc[date:date_end]['Gd(i)'].plot(legend='Gd(i)')
timeseries.loc[date:date_end]['Gr(i)'].plot(legend='Gr(i)')
plt.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Yearly PV production

In [131]:
annual_production = timeseries.P.sum()/1000 # kWh
annual_production

30551.31932

### Yearly equivalent hours

In [125]:
annual_equivalent_hours = annual_production/peakpower # [kWh/kW] or [h]
annual_equivalent_hours

1335.3755239520956